In [130]:
import numpy as np

In [131]:
import pandas as pd
import os
import glob
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [132]:
# Initialize an empty dictionary to hold all data
data_dict = {}

# Define a set to keep track of all unique SNP positions
snp_positions = set()

# Define the path to the directory containing the edited VCF files
edited_vcf_dir = r"/mnt/raid0/michalis/snakemakeproject/edited_vcfs/"


In [ ]:
print(edited_vcf_dir)

In [134]:
vcf_files =os.listdir(edited_vcf_dir)

In [ ]:
vcf_files[0]

In [136]:
file_path = vcf_files[0]
unique_id = os.path.join(edited_vcf_dir,file_path)
#print(unique_id)
snp_positions=[]
genotypes=[]
with open(unique_id, 'r') as f:
    for line in f:
        line = line.strip().replace('\r', '')  # Strip newline and remove carriage return
        if not line:
            continue  # skip empty lines
        parts = line.split()
        snp_position = int(parts[0])
        genotype = parts[1]
        snp_positions.append(snp_position)
        genotypes.append(genotype)
        #genotypes = [-1 if x == '.' else x for x in genotypes]
        #genotypes = [int(x) for x in genotypes]
df = pd.DataFrame({'SNPs' : snp_positions, file_path: genotypes})
# Replace '.' with 0 in column 'A'
df[file_path] = df[file_path].replace('.', 0)
#df[file_path] = df[file_path].replace({val: 0 for val in df[file_path].unique() if val not in [0, 1]})
# Convert values in column 'A' to integers
df[file_path] = df[file_path].astype(int)
df[file_path] = df[file_path].replace({val: 1 for val in df[file_path].unique() if val not in [0, 1]})
sdf = df.astype(pd.SparseDtype(int, fill_value=0))

In [ ]:
sdf

In [ ]:
count=0
for file_path in vcf_files[1:12261]:
    unique_id = os.path.join(edited_vcf_dir,file_path)
    #print(unique_id)
    count+=1
    snp_positions=[]
    genotypes=[]
    print(count)
    with open(unique_id, 'r') as f:
        for line in f:
            line = line.strip().replace('\r', '')  # Strip newline and remove carriage return
            if not line:
                continue  # skip empty lines
            parts = line.split()
            genotype = parts[1]
            genotypes.append(genotype)
            #genotypes = [-1 if x == '.' else x for x in genotypes]
            #genotypes = [int(x) for x in genotypes]
    sdf[file_path] = genotypes
    # Replace '.' with -1 in column 'A'
    # Replace all values except 0 or 1 with 0
    sdf[file_path] = sdf[file_path].replace('.', 0)
    # Convert values in column 'A' to integers
    sdf[file_path] = sdf[file_path].astype(int)
    # Replace all values except 0 or 1 with 0
    sdf[file_path] = sdf[file_path].replace({val: 1 for val in sdf[file_path].unique() if val not in [0, 1]})

    

    sdf = sdf.astype(pd.SparseDtype(int, fill_value=0))

In [ ]:
sdf

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, vstack

# Define chunk size 
chunk_size = 1000

# Initialize a list to store chunks of the sparse matrix
array_ML_chunks = []

# Iterate over columns in chunks, starting from column 1 (index 0 is skipped)
for start_col in range(1, sdf.shape[1], chunk_size):  # Process all columns of sdf
    end_col = min(start_col + chunk_size, sdf.shape[1])

    # Select and transpose the current chunk
    df_transposed_chunk = sdf.iloc[:, start_col:end_col].T

    # Convert chunk to sparse matrix
    array_ML_chunk = csr_matrix(df_transposed_chunk.values)

    # Append the sparse matrix chunk to the list
    array_ML_chunks.append(array_ML_chunk)

    # Print a progress message after processing each chunk
    print(f"Processed chunk {len(array_ML_chunks)} (columns {start_col}-{end_col - 1})")

# Combine chunks into a single sparse matrix
array_ML_sparse = vstack(array_ML_chunks, format='csr')

# Now array_ML.shape should be (12261, 1265713)
print(array_ML_sparse .shape)



In [ ]:
print(array_ML_sparse .shape)

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

# Read the sdf file which is assumed to be already loaded into a variable 'sdf'
columns_sdf = sdf.iloc[:, 1:12262].columns
columns_sdf = np.array([s[:-4] for s in columns_sdf])

# Assuming array_ML_sparse is your sparse genotype data matrix
print(f"Original shape of array_ML_sparse: {array_ML_sparse.shape}")

# Sum the columns to find columns with only zeros
column_sums = np.array(array_ML_sparse.sum(axis=0)).flatten()

# Identify columns that have at least one non-zero entry
non_zero_columns = column_sums > 0

# Filter the columns in array_ML_sparse
array_ML_sparse_filtered = array_ML_sparse[:, non_zero_columns]

print(f"New shape of array_ML_sparse: {array_ML_sparse_filtered.shape}")

# Function to process the data for each antibiotic
def process_data(metadata_file, phenotype_column):
    # Read the phenotypes file 
    metadata = pd.read_csv(metadata_file)

    # Check if all IDs in columns_sdf are in metadata['UNIQUEID']
    if not set(columns_sdf).issubset(set(metadata['UNIQUEID'])):
        raise ValueError("Not all patient IDs in the feature matrix are present in the metadata file.")

    # Extract target values based on UNIQUEID
    phenotype_values = metadata.set_index('UNIQUEID').loc[columns_sdf, phenotype_column]

    # Convert the extracted phenotype_values to a numpy array
    phenotype_values = phenotype_values.values

    # Convert array to str
    phenotype_values = phenotype_values.astype('str')

    # Find indices where values are 'S' or 'R'
    valid_indices = (phenotype_values == 'S') | (phenotype_values == 'R')

    # Filter out 'I' and NaN values
    y = phenotype_values[valid_indices]
    X = array_ML_sparse_filtered[valid_indices]

    # Replace 'S' with 0 and 'R' with 1
    y[y == 'S'] = 0
    y[y == 'R'] = 1
    y = y.astype(int)

    return X, y

# Process data for each antibiotic
X_emb, y_emb = process_data("Cryptic_Ethambutol_Filtered.csv", 'EMB_BINARY_PHENOTYPE')
X_inh, y_inh = process_data("Cryptic_Isoniazid_Filtered.csv", 'INH_BINARY_PHENOTYPE')
X_rif, y_rif = process_data("Cryptic_Rifampicin_Filtered.csv", 'RIF_BINARY_PHENOTYPE')

# The resulting X_emb, y_emb, X_inh, y_inh, X_rif, and y_rif are now filtered and converted correctly


In [ ]:
print(X_rif.shape)

In [ ]:
print(X_emb.shape)

In [ ]:
print(X_inh.shape)

In [ ]:
print(y_emb.shape)

In [ ]:
print(y_inh.shape)

In [ ]:
print(y_rif.shape)

In [ ]:
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from scipy.sparse import csr_matrix

# Assuming X_inh, X_emb, and X_rif are your filtered genotype data matrices in CSR format

# Variance thresholding function
def variance_threshold_reduction(X, threshold=0.01):
    print(f"Original shape of X: {X.shape}")
    selector = VarianceThreshold(threshold=threshold)
    X_reduced = selector.fit_transform(X)
    print(f"Shape of X after variance thresholding: {X_reduced.shape}")
    return X_reduced

# Apply the variance thresholding to X_inh
X_inh_reduced = variance_threshold_reduction(X_inh, threshold=0.01)

# Apply the variance thresholding to X_emb
X_emb_reduced = variance_threshold_reduction(X_emb, threshold=0.01)

# Apply the variance thresholding to X_rif
X_rif_reduced = variance_threshold_reduction(X_rif, threshold=0.01)


In [ ]:
print(X_inh_reduced.shape)

In [ ]:
print(X_emb_reduced.shape)

In [ ]:
print(X_rif_reduced.shape)

In [153]:
#SVM:

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, make_scorer, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from scipy import sparse
import pandas as pd

# Assuming X_inh, y_inh, X_emb, y_emb, X_rif, y_rif are already defined and loaded
# Assuming X_inh_reduced, X_emb_reduced, X_rif_reduced are already defined

# Function to train and evaluate SVM with different hyperparameters
def evaluate_svm(X, y):
    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

    # Feature scaling
    scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Define the parameter grid
    param_grid = {
        'C': {0.1, 1, 10},
        'kernel': {'linear', 'rbf'}
    }

    # Initialize SVM classifier 
    svm_model = SVC(probability=True, random_state=1, verbose=False)

    # Initialize GridSearchCV with reduced n_jobs
    grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=4, verbose=0)

    # Fit the model
    grid_search.fit(X_train, y_train)

    # Get the best model
    best_svm_model = grid_search.best_estimator_

    # Make predictions
    y_pred = best_svm_model.predict_proba(X_test)[:, 1]  # Get probabilities for positive class
    predictions = [round(value) for value in y_pred]

    # Calculate sensitivity and specificity
    def sensitivity_specificity_score(y_true, y_pred):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        return sensitivity, specificity

    # Print test set metrics
    print(f"Test Set Accuracy (C={C_value}):", accuracy_score(y_test, predictions))
    print(f"Test Set F1-Score (C={C_value}):", f1_score(y_test, predictions))
    print(f"Test Set ROC-AUC (C={C_value}):", roc_auc_score(y_test, y_pred))

    sensitivity, specificity = sensitivity_specificity_score(y_test, predictions)
    print(f"Test Set Sensitivity (C={C_value}): {sensitivity:.3f}")
    print(f"Test Set Specificity (C={C_value}): {specificity:.3f}")

    # Print the best parameters
    print(f"Best parameters found: {grid_search.best_params_}")
# List of datasets and their corresponding labels
datasets = [
    (X_inh, y_inh, "X_inh"),
    (X_emb, y_emb, "X_emb"),
    (X_rif, y_rif, "X_rif"),
    (X_inh_reduced, y_inh, "X_inh_reduced"),
    (X_emb_reduced, y_emb, "X_emb_reduced"),
    (X_rif_reduced, y_rif, "X_rif_reduced")
]

# Evaluate SVM with different C values for each dataset
C_values = [1]
for X, y, label in datasets:
    print(f"Evaluating dataset: {label}")
    for C in C_values:
        print(f"Evaluating SVM with C={C}")
        evaluate_svm(X, y, C)
        print("\n")

In [33]:
C_value =1 

# SVM INH

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_inh, y_inh, test_size=0.2, random_state=1, stratify=y_inh)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize SVM classifier with Linear kernel for faster computation
svm_inh = SVC(kernel='linear', C=1, probability=True, random_state=1, verbose=False)

# Fit the model
svm_inh.fit(X_train, y_train)

# Make predictions
y_pred = svm_inh.predict_proba(X_test)[:, 1]  # Get probabilities for positive class
predictions = [round(value) for value in y_pred]

# Calculate sensitivity and specificity
def sensitivity_specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    return sensitivity, specificity

# Print test set metrics
print(f"Test Set Accuracy (C={C_value}):", accuracy_score(y_test, predictions))
print(f"Test Set F1-Score (C={C_value}):", f1_score(y_test, predictions))
print(f"Test Set ROC-AUC (C={C_value}):", roc_auc_score(y_test, y_pred))

sensitivity, specificity = sensitivity_specificity_score(y_test, predictions)
print(f"Test Set Sensitivity (C={C_value}): {sensitivity:.3f}")
print(f"Test Set Specificity (C={C_value}): {specificity:.3f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_inh, y_inh, test_size=0.2, random_state=1, stratify=y_inh)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = svm_inh.predict(X_test)
y_pred_prob = svm_inh.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/svm/inh/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/svm/inh/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/svm/inh/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/svm/inh/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/svm/inh/predicted_probability_distributions.png')
plt.close()

# SVM emb

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_emb, y_emb, test_size=0.2, random_state=1, stratify=y_emb)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize SVM classifier with Linear kernel for faster computation
svm_emb = SVC(kernel='linear', C=1, probability=True, random_state=1, verbose=False)

# Fit the model
svm_emb.fit(X_train, y_train)

# Make predictions
y_pred = svm_emb.predict_proba(X_test)[:, 1]  # Get probabilities for positive class
predictions = [round(value) for value in y_pred]

# Calculate sensitivity and specificity
def sensitivity_specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    return sensitivity, specificity

# Print test set metrics
print(f"Test Set Accuracy (C={C_value}):", accuracy_score(y_test, predictions))
print(f"Test Set F1-Score (C={C_value}):", f1_score(y_test, predictions))
print(f"Test Set ROC-AUC (C={C_value}):", roc_auc_score(y_test, y_pred))

sensitivity, specificity = sensitivity_specificity_score(y_test, predictions)
print(f"Test Set Sensitivity (C={C_value}): {sensitivity:.3f}")
print(f"Test Set Specificity (C={C_value}): {specificity:.3f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_emb, y_emb, test_size=0.2, random_state=1, stratify=y_emb)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = svm_emb.predict(X_test)
y_pred_prob = svm_emb.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/svm/emb/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/svm/emb/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/svm/emb/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/svm/emb/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/svm/emb/predicted_probability_distributions.png')
plt.close()

# SVM rif

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_rif, y_rif, test_size=0.2, random_state=1, stratify=y_rif)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize SVM classifier with Linear kernel for faster computation
svm_rif = SVC(kernel='linear', C=1, probability=True, random_state=1, verbose=False)

# Fit the model
svm_rif.fit(X_train, y_train)

# Make predictions
y_pred = svm_rif.predict_proba(X_test)[:, 1]  # Get probabilities for positive class
predictions = [round(value) for value in y_pred]

# Calculate sensitivity and specificity
def sensitivity_specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    return sensitivity, specificity

# Print test set metrics
print(f"Test Set Accuracy (C={C_value}):", accuracy_score(y_test, predictions))
print(f"Test Set F1-Score (C={C_value}):", f1_score(y_test, predictions))
print(f"Test Set ROC-AUC (C={C_value}):", roc_auc_score(y_test, y_pred))

sensitivity, specificity = sensitivity_specificity_score(y_test, predictions)
print(f"Test Set Sensitivity (C={C_value}): {sensitivity:.3f}")
print(f"Test Set Specificity (C={C_value}): {specificity:.3f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_rif, y_rif, test_size=0.2, random_state=1, stratify=y_rif)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = svm_rif.predict(X_test)
y_pred_prob = svm_rif.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/svm/rif/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/svm/rif/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/svm/rif/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/svm/rif/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/svm/rif/predicted_probability_distributions.png')
plt.close()

# SVM inh r

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_inh_reduced, y_inh, test_size=0.2, random_state=1, stratify=y_inh)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize SVM classifier with Linear kernel for faster computation
svm_inh_reduced = SVC(kernel='linear', C=1, probability=True, random_state=1, verbose=False)

# Fit the model
svm_inh_reduced.fit(X_train, y_train)

# Make predictions
y_pred = svm_inh_reduced.predict_proba(X_test)[:, 1]  # Get probabilities for positive class
predictions = [round(value) for value in y_pred]

# Calculate sensitivity and specificity
def sensitivity_specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    return sensitivity, specificity

# Print test set metrics
print(f"Test Set Accuracy (C={C_value}):", accuracy_score(y_test, predictions))
print(f"Test Set F1-Score (C={C_value}):", f1_score(y_test, predictions))
print(f"Test Set ROC-AUC (C={C_value}):", roc_auc_score(y_test, y_pred))

sensitivity, specificity = sensitivity_specificity_score(y_test, predictions)
print(f"Test Set Sensitivity (C={C_value}): {sensitivity:.3f}")
print(f"Test Set Specificity (C={C_value}): {specificity:.3f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_inh_reduced, y_inh, test_size=0.2, random_state=1, stratify=y_inh)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = svm_inh_reduced.predict(X_test)
y_pred_prob = svm_inh_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/svm/inh_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/svm/inh_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/svm/inh_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/svm/inh_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/svm/inh_r/predicted_probability_distributions.png')
plt.close()

# SVM emb r

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_emb_reduced, y_emb, test_size=0.2, random_state=1, stratify=y_emb)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize SVM classifier with Linear kernel for faster computation
svm_emb_reduced = SVC(kernel='linear', C=1, probability=True, random_state=1, verbose=False)

# Fit the model
svm_emb_reduced.fit(X_train, y_train)

# Make predictions
y_pred = svm_emb_reduced.predict_proba(X_test)[:, 1]  # Get probabilities for positive class
predictions = [round(value) for value in y_pred]

# Calculate sensitivity and specificity
def sensitivity_specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    return sensitivity, specificity

# Print test set metrics
print(f"Test Set Accuracy (C={C_value}):", accuracy_score(y_test, predictions))
print(f"Test Set F1-Score (C={C_value}):", f1_score(y_test, predictions))
print(f"Test Set ROC-AUC (C={C_value}):", roc_auc_score(y_test, y_pred))

sensitivity, specificity = sensitivity_specificity_score(y_test, predictions)
print(f"Test Set Sensitivity (C={C_value}): {sensitivity:.3f}")
print(f"Test Set Specificity (C={C_value}): {specificity:.3f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_emb_reduced, y_emb, test_size=0.2, random_state=1, stratify=y_emb)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = svm_emb_reduced.predict(X_test)
y_pred_prob = svm_emb_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/svm/emb_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/svm/emb_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/svm/emb_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/svm/emb_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/svm/emb_r/predicted_probability_distributions.png')
plt.close()

# SVM rif r

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_rif_reduced, y_rif, test_size=0.2, random_state=1, stratify=y_rif)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize SVM classifier with Linear kernel for faster computation
svm_rif_reduced = SVC(kernel='linear', C=1, probability=True, random_state=1, verbose=False)

# Fit the model
svm_rif_reduced.fit(X_train, y_train)

# Make predictions
y_pred = svm_rif_reduced.predict_proba(X_test)[:, 1]  # Get probabilities for positive class
predictions = [round(value) for value in y_pred]

# Calculate sensitivity and specificity
def sensitivity_specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    return sensitivity, specificity

# Print test set metrics
print(f"Test Set Accuracy (C={C_value}):", accuracy_score(y_test, predictions))
print(f"Test Set F1-Score (C={C_value}):", f1_score(y_test, predictions))
print(f"Test Set ROC-AUC (C={C_value}):", roc_auc_score(y_test, y_pred))

sensitivity, specificity = sensitivity_specificity_score(y_test, predictions)
print(f"Test Set Sensitivity (C={C_value}): {sensitivity:.3f}")
print(f"Test Set Specificity (C={C_value}): {specificity:.3f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_rif_reduced, y_rif, test_size=0.2, random_state=1, stratify=y_rif)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = svm_rif_reduced.predict(X_test)
y_pred_prob = svm_rif_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/svm/rif_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/svm/rif_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/svm/rif_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/svm/rif_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/svm/rif_r/predicted_probability_distributions.png')
plt.close()

In [ ]:
#RF WITH HYPERPARAMETER TUNING:

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, make_scorer, confusion_matrix
from sklearn.preprocessing import StandardScaler
from scipy import sparse
import pandas as pd

# Assuming X_inh, y_inh, X_emb, y_emb, X_rif, y_rif are already defined and loaded
# Assuming X_inh_reduced, X_emb_reduced, X_rif_reduced are already defined

# List of datasets and their corresponding labels
datasets = [
    (X_inh, y_inh, "X_inh"),
    (X_emb, y_emb, "X_emb"),
    (X_rif, y_rif, "X_rif"),
    (X_inh_reduced, y_inh, "X_inh_reduced"),
    (X_emb_reduced, y_emb, "X_emb_reduced"),
    (X_rif_reduced, y_rif, "X_rif_reduced")
]

# Dictionary to store the best models for each dataset
best_models = {}

for X, y, label in datasets:
    print(f"Evaluating dataset: {label}")

    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

    # Feature scaling
    scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Define the parameter grid
    param_grid = {
        'n_estimators': [100, 200],
        'max_features': ['sqrt', 'log2'],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'bootstrap': [True, False]
    }

    # Initialize Random Forest classifier
    rf_model = RandomForestClassifier(random_state=1)

    # Initialize GridSearchCV with reduced n_jobs
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=4, verbose=0)

    # Fit the model
    grid_search.fit(X_train, y_train)

    # Get the best model
    best_rf_model = grid_search.best_estimator_

    # Store the best model in the dictionary
    best_models[label] = best_rf_model

    # Make predictions
    y_pred = best_rf_model.predict_proba(X_test)[:, 1]  # Get probabilities for positive class
    predictions = [round(value) for value in y_pred]

    # Calculate sensitivity and specificity
    def sensitivity_specificity_score(y_true, y_pred):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        return sensitivity, specificity

    # Print test set metrics
    print("Test Set Accuracy:", accuracy_score(y_test, predictions))
    print("Test Set F1-Score:", f1_score(y_test, predictions))
    print("Test Set ROC-AUC:", roc_auc_score(y_test, y_pred))

    sensitivity, specificity = sensitivity_specificity_score(y_test, predictions)
    print(f"Test Set Sensitivity: {sensitivity:.3f}")
    print(f"Test Set Specificity: {specificity:.3f}")

    # Print the best parameters
    print(f"Best parameters found: {grid_search.best_params_}")
    print("\n")

# best_models dictionary now contains the best model for each dataset
for label, model in best_models.items():
    print(f"Best model for {label} is stored.")

In [54]:
rf_inh = best_models['X_inh']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_inh, y_inh, test_size=0.2, random_state=1, stratify=y_inh)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = rf_inh.predict(X_test)
y_pred_prob = rf_inh.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/rf/inh/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/rf/inh/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/rf/inh/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/rf/inh/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/rf/inh/predicted_probability_distributions.png')
plt.close()

In [78]:
rf_emb = best_models['X_emb']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_emb, y_emb, test_size=0.2, random_state=1, stratify=y_emb)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = rf_emb.predict(X_test)
y_pred_prob = rf_emb.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/rf/emb/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/rf/emb/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/rf/emb/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/rf/emb/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/rf/emb/predicted_probability_distributions.png')
plt.close()

In [80]:
rf_rif = best_models['X_rif']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_rif, y_rif, test_size=0.2, random_state=1, stratify=y_rif)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = rf_rif.predict(X_test)
y_pred_prob = rf_rif.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/rf/rif/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/rf/rif/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/rf/rif/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/rf/rif/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/rf/rif/predicted_probability_distributions.png')
plt.close()

In [83]:
rf_inh_reduced = best_models['X_inh_reduced']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_inh_reduced, y_inh, test_size=0.2, random_state=1, stratify=y_inh)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = rf_inh_reduced.predict(X_test)
y_pred_prob = rf_inh_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/rf/inh_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/rf/inh_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/rf/inh_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/rf/inh_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/rf/inh_r/predicted_probability_distributions.png')
plt.close()

In [85]:
rf_emb_reduced = best_models['X_emb_reduced']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_emb_reduced, y_emb, test_size=0.2, random_state=1, stratify=y_emb)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = rf_emb_reduced.predict(X_test)
y_pred_prob = rf_emb_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/rf/emb_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/rf/emb_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/rf/emb_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/rf/emb_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/rf/emb_r/predicted_probability_distributions.png')
plt.close()

In [87]:
rf_rif_reduced = best_models['X_rif_reduced']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_rif_reduced, y_rif, test_size=0.2, random_state=1, stratify=y_rif)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = rf_rif_reduced.predict(X_test)
y_pred_prob = rf_rif_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/rf/rif_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/rf/rif_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/rf/rif_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/rf/rif_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/rf/rif_r/predicted_probability_distributions.png')
plt.close()

In [ ]:
#LOGISTIC REGRESSION WITH L2 REGULARIZATION:

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, make_scorer, confusion_matrix
from scipy import sparse
from sklearn.feature_selection import SelectKBest, chi2

best_modelslr = {}

# Function to perform feature selection and evaluate Logistic Regression for a given dataset
def evaluate_log_reg_with_feature_selection(X, y):
    # Feature selection or dimensionality reduction (example: selecting top 1000 features)
    if not sparse.issparse(X):
        selector = SelectKBest(chi2, k=1000)
        X = selector.fit_transform(X, y)

    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

    # Feature scaling
    if sparse.issparse(X):
        scaler = StandardScaler(with_mean=False)
    else:
        scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train).toarray() if sparse.issparse(X_train) else scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test).toarray() if sparse.issparse(X_test) else scaler.transform(X_test)

    # Define the parameter grid
    param_grid = {
        'solver': {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'},
        'penalty': {None, 'l1', 'l2', 'elasticnet'},
        'C': {100, 10, 1.0, 0.1, 0.01}
    }

    # Initialize Logistic Regression with L2 regularization and smaller max_iter
    lr = LogisticRegression(prandom_state=1)

    # Initialize GridSearchCV with reduced n_jobs
    grid_search = GridSearchCV(estimator=lr, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=4, verbose=0)

    # Fit the model
    grid_search.fit(X_train, y_train)

    # Get the best model
    log_reg = grid_search.best_estimator_

    best_modelslr{label} = log_reg
    
    # Make predictions
    y_pred_proba = log_reg.predict_proba(X_test)[:, 1]  # Get probabilities for positive class
    y_pred = log_reg.predict(X_test)

    # Calculate sensitivity and specificity
    def sensitivity_specificity_score(y_true, y_pred):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        return sensitivity, specificity


    # Print test set metrics
    print("Test Set Accuracy:", accuracy_score(y_test, y_pred))
    print("Test Set F1-Score:", f1_score(y_test, y_pred))
    print("Test Set ROC-AUC:", roc_auc_score(y_test, y_pred_proba))

    sensitivity, specificity = sensitivity_specificity_score(y_test, y_pred)
    print(f"Test Set Sensitivity: {sensitivity:.3f}")
    print(f"Test Set Specificity: {specificity:.3f}")

# List of datasets and their corresponding labels
datasets = [
    (X_inh, y_inh, "X_inh"),
    (X_emb, y_emb, "X_emb"),
    (X_rif, y_rif, "X_rif"),
    (X_inh_reduced, y_inh, "X_inh_reduced"),
    (X_emb_reduced, y_emb, "X_emb_reduced"),
    (X_rif_reduced, y_rif, "X_rif_reduced")
]

# Evaluate Logistic Regression with feature selection for each dataset
for X, y, label in datasets:
    print(f"Evaluating dataset: {label}")
    evaluate_log_reg_with_feature_selection(X, y)
    print("\n")

In [121]:
lr_inh_reduced = best_modelslr['X_inh_reduced']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_inh_reduced, y_inh, test_size=0.2, random_state=1, stratify=y_inh)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = lr_inh_reduced.predict(X_test)
y_pred_prob = lr_inh_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/lr/inh_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/lr/inh_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/lr/inh_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/lr/inh_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/lr/inh_r/predicted_probability_distributions.png')
plt.close()

In [124]:
lr_emb_reduced = best_modelslr['X_emb_reduced']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_emb_reduced, y_emb, test_size=0.2, random_state=1, stratify=y_emb)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = lr_emb_reduced.predict(X_test)
y_pred_prob = lr_emb_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/lr/emb_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/lr/emb_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/lr/emb_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/lr/emb_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/lr/emb_r/predicted_probability_distributions.png')
plt.close()

In [ ]:
#XGBOOST WITH HYPERPARAMETER TUNING:

In [155]:
lr_rif_reduced = best_modelslr['X_rif_reduced']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_rif_reduced, y_rif, test_size=0.2, random_state=1, stratify=y_rif)

# Feature scaling
scaler = StandardScaler(with_mean=False) if sparse.issparse(X) else StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predict the labels
y_pred = lr_rif_reduced.predict(X_test)
y_pred_prob = lr_rif_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/lr/rif_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/lr/rif_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/lr/rif_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/lr/rif_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/lr/rif_r/predicted_probability_distributions.png')
plt.close()

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, make_scorer, confusion_matrix
import pandas as pd

# Assuming X_inh, y_inh, X_emb, y_emb, X_rif, y_rif are already defined and loaded
# Assuming X_inh_reduced, X_emb_reduced, X_rif_reduced are already defined

# List of datasets and their corresponding labels
datasets = [
    (X_inh, y_inh, "X_inh"),
    (X_emb, y_emb, "X_emb"),
    (X_rif, y_rif, "X_rif"),
    (X_inh_reduced, y_inh, "X_inh_reduced"),
    (X_emb_reduced, y_emb, "X_emb_reduced"),
    (X_rif_reduced, y_rif, "X_rif_reduced")
]

# Dictionary to store the best models for each dataset
best_modelsxg = {}

for X, y, label in datasets:
    print(f"Evaluating dataset: {label}")

    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

    # Define the initial XGBoost parameters
    params = {
        'objective': 'binary:logistic',  
        'random_state': 1         
    }

    # Define the parameter grid to search
    param_distributions = {
        'eta': [0.01, 0.1, 0.2],
        'max_depth': [4, 6, 8, 10],
        'subsample': [0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
        'n_estimators': [50, 100, 200]
    }

    # Initialize XGBClassifier with initial params
    model = xgb.XGBClassifier(**params)

    # Create the scoring dictionary
    scoring = {
        'accuracy': make_scorer(accuracy_score), 
        'f1': make_scorer(f1_score), 
        'roc_auc': make_scorer(roc_auc_score)
    }

    # Use RandomizedSearchCV to find the best parameters
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, scoring='roc_auc', cv=5, n_jobs=4, n_iter=10, verbose=0, random_state=1)
    random_search.fit(X_train, y_train)

    # Get the best parameters from the random search
    best_params = random_search.best_params_
    print("Best Parameters found: ", best_params)

    # Initialize XGBClassifier with best parameters
    best_model = xgb.XGBClassifier(**best_params)

    # Fit the model with the best parameters
    best_model.fit(X_train, y_train)

    # Store the best model in the dictionary
    best_modelsxg[label] = best_model

    # Make predictions
    y_pred = best_model.predict_proba(X_test)[:, 1]  # Get probabilities for positive class
    predictions = [round(value) for value in y_pred]

    # Calculate sensitivity and specificity
    def sensitivity_specificity_score(y_true, y_pred):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        return sensitivity, specificity

    # Print test set metrics
    print("Test Set Accuracy:", accuracy_score(y_test, predictions))
    print("Test Set F1-Score:", f1_score(y_test, predictions))
    print("Test Set ROC-AUC:", roc_auc_score(y_test, y_pred))

    sensitivity, specificity = sensitivity_specificity_score(y_test, predictions)
    print(f"Test Set Sensitivity: {sensitivity:.3f}")
    print(f"Test Set Specificity: {specificity:.3f}")

    print("\n")

# best_models dictionary now contains the best model for each dataset
for label, model in best_modelsxg.items():
    print(f"Best model for {label} is stored.")


In [90]:
xg_inh = best_modelsxg['X_inh']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_inh, y_inh, test_size=0.2, random_state=1, stratify=y_inh)

# Predict the labels
y_pred = xg_inh.predict(X_test)
y_pred_prob = xg_inh.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/xg/inh/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/xg/inh/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/xg/inh/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/xg/inh/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/xg/inh/predicted_probability_distributions.png')
plt.close()

In [94]:
xg_emb = best_modelsxg['X_emb']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_emb, y_emb, test_size=0.2, random_state=1, stratify=y_emb)


# Predict the labels
y_pred = xg_emb.predict(X_test)
y_pred_prob = xg_emb.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/xg/emb/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/xg/emb/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/xg/emb/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/xg/emb/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/xg/emb/predicted_probability_distributions.png')
plt.close()

In [96]:
xg_rif = best_modelsxg['X_rif']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_rif, y_rif, test_size=0.2, random_state=1, stratify=y_rif)


# Predict the labels
y_pred = xg_rif.predict(X_test)
y_pred_prob = xg_rif.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/xg/rif/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/xg/rif/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/xg/rif/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/xg/rif/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/xg/rif/predicted_probability_distributions.png')
plt.close()

In [98]:
xg_inh_reduced = best_modelsxg['X_inh_reduced']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_inh_reduced, y_inh, test_size=0.2, random_state=1, stratify=y_inh)

# Predict the labels
y_pred = xg_inh_reduced.predict(X_test)
y_pred_prob = xg_inh_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/xg/inh_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/xg/inh_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/xg/inh_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/xg/inh_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/xg/inh_r/predicted_probability_distributions.png')
plt.close()

In [101]:
xg_emb_reduced = best_modelsxg['X_emb_reduced']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_emb_reduced, y_emb, test_size=0.2, random_state=1, stratify=y_emb)

# Predict the labels
y_pred = xg_emb_reduced.predict(X_test)
y_pred_prob = xg_emb_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/xg/emb_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/xg/emb_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/xg/emb_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/xg/emb_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/xg/emb_r/predicted_probability_distributions.png')
plt.close()

In [159]:
xg_rif_reduced = best_modelsxg['X_rif_reduced']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_rif_reduced, y_rif, test_size=0.2, random_state=1, stratify=y_rif)

# Predict the labels
y_pred = xg_rif_reduced.predict(X_test)
y_pred_prob = xg_rif_reduced.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/xg/rif_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/xg/rif_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/xg/rif_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/xg/rif_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/xg/rif_r/predicted_probability_distributions.png')
plt.close()

In [ ]:
#CNNs:

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from scipy import sparse
from keras import backend as K

# Function to train and evaluate a CNN for a given dataset
def evaluate_cnn(X, y):
    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

    # Feature scaling
    if sparse.issparse(X):
        scaler = StandardScaler(with_mean=False)
    else:
        scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train).toarray() if sparse.issparse(X_train) else scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test).toarray() if sparse.issparse(X_test) else scaler.transform(X_test)

    # Reshape data for Conv1D layer
    X_train = X_train[..., np.newaxis]  # Add a new axis for the single channel
    X_test = X_test[..., np.newaxis]

    # Define the CNN model using Input layer
    model = Sequential([
        Input(shape=(X_train.shape[1], 1)),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Define early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

    # Train the model with a smaller batch size
    history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, 
                        callbacks=[early_stopping, reduce_lr], verbose=0)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Set Accuracy: {accuracy:.3f}")

    # Make predictions
    y_pred_proba = model.predict(X_test).flatten()
    y_pred = (y_pred_proba >= 0.5).astype(int)

    # Calculate additional metrics
    print("Test Set Accuracy:", accuracy_score(y_test, y_pred))
    print("Test Set F1-Score:", f1_score(y_test, y_pred))
    print("Test Set ROC-AUC:", roc_auc_score(y_test, y_pred_proba))

    def sensitivity_specificity_score(y_true, y_pred):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        return sensitivity, specificity

    sensitivity, specificity = sensitivity_specificity_score(y_test, y_pred)
    print(f"Test Set Sensitivity: {sensitivity:.3f}")
    print(f"Test Set Specificity: {specificity:.3f}")

    # Clear the Keras session to free up memory
    K.clear_session()

# List of reduced datasets and their corresponding labels
reduced_datasets = [
    (X_inh_reduced, y_inh, "X_inh_reduced"),
    (X_emb_reduced, y_emb, "X_emb_reduced"),
    (X_rif_reduced, y_rif, "X_rif_reduced")
]

# Evaluate CNN for each reduced dataset
for X, y, label in reduced_datasets:
    print(f"Evaluating dataset: {label}")
    evaluate_cnn(X, y)
    print("\n")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from scipy import sparse
from keras import backend as K

# Function to train and evaluate a CNN for a given dataset
def evaluate_cnn(X, y):
    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

    # Feature scaling
    if sparse.issparse(X):
        scaler = StandardScaler(with_mean=False)
    else:
        scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train).toarray() if sparse.issparse(X_train) else scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test).toarray() if sparse.issparse(X_test) else scaler.transform(X_test)

    # Reshape data for Conv1D layer
    X_train = X_train[..., np.newaxis]  # Add a new axis for the single channel
    X_test = X_test[..., np.newaxis]

    # Define the CNN model using Input layer
    model = Sequential([
        Input(shape=(X_train.shape[1], 1)),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Define early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

    # Train the model with a smaller batch size
    history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, 
                        callbacks=[early_stopping, reduce_lr], verbose=0)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Set Accuracy: {accuracy:.3f}")

    # Make predictions
    y_pred_proba = model.predict(X_test).flatten()
    y_pred = (y_pred_proba >= 0.5).astype(int)

    # Calculate additional metrics
    print("Test Set Accuracy:", accuracy_score(y_test, y_pred))
    print("Test Set F1-Score:", f1_score(y_test, y_pred))
    print("Test Set ROC-AUC:", roc_auc_score(y_test, y_pred_proba))

    def sensitivity_specificity_score(y_true, y_pred):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        return sensitivity, specificity

    sensitivity, specificity = sensitivity_specificity_score(y_test, y_pred)
    print(f"Test Set Sensitivity: {sensitivity:.3f}")
    print(f"Test Set Specificity: {specificity:.3f}")

    # Clear the Keras session to free up memory
    K.clear_session()

# List of reduced datasets and their corresponding labels
reduced_datasets = [
    (X_inh_reduced, y_inh, "X_inh_reduced"),
    (X_emb_reduced, y_emb, "X_emb_reduced"),
    (X_rif_reduced, y_rif, "X_rif_reduced")
]

# Evaluate CNN for each reduced dataset
for X, y, label in reduced_datasets:
    print(f"Evaluating dataset: {label}")
    evaluate_cnn(X, y)
    print("\n")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from scipy import sparse
from keras import backend as K

# Function to train and evaluate a CNN for a given dataset
def evaluate_cnn(X, y):
    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

    # Feature scaling
    if sparse.issparse(X):
        scaler = StandardScaler(with_mean=False)
    else:
        scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train).toarray() if sparse.issparse(X_train) else scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test).toarray() if sparse.issparse(X_test) else scaler.transform(X_test)

    # Reshape data for Conv1D layer
    X_train = X_train[..., np.newaxis]  # Add a new axis for the single channel
    X_test = X_test[..., np.newaxis]

    # Define the CNN model using Input layer
    model = Sequential([
        Input(shape=(X_train.shape[1], 1)),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Define early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

    # Train the model with a smaller batch size
    history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, 
                        callbacks=[early_stopping, reduce_lr], verbose=0)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Set Accuracy: {accuracy:.3f}")

    # Make predictions
    y_pred_proba = model.predict(X_test).flatten()
    y_pred = (y_pred_proba >= 0.5).astype(int)

    # Calculate additional metrics
    print("Test Set Accuracy:", accuracy_score(y_test, y_pred))
    print("Test Set F1-Score:", f1_score(y_test, y_pred))
    print("Test Set ROC-AUC:", roc_auc_score(y_test, y_pred_proba))

    def sensitivity_specificity_score(y_true, y_pred):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        return sensitivity, specificity

    sensitivity, specificity = sensitivity_specificity_score(y_test, y_pred)
    print(f"Test Set Sensitivity: {sensitivity:.3f}")
    print(f"Test Set Specificity: {specificity:.3f}")

    # Clear the Keras session to free up memory
    K.clear_session()

# List of reduced datasets and their corresponding labels
reduced_datasets = [
    (X_inh_reduced, y_inh, "X_inh_reduced"),
    (X_emb_reduced, y_emb, "X_emb_reduced"),
    (X_rif_reduced, y_rif, "X_rif_reduced")
]

# Evaluate CNN for each reduced dataset
for X, y, label in reduced_datasets:
    print(f"Evaluating dataset: {label}")
    evaluate_cnn(X, y)
    print("\n")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from scipy import sparse
from keras import backend as K

# Assuming X_inh_reduced, y_inh, X_emb_reduced, y_emb, X_rif_reduced, y_rif are already defined and loaded

# List of reduced datasets and their corresponding labels
reduced_datasets = [
    (X_inh_reduced, y_inh, "X_inh_reduced"),
    (X_emb_reduced, y_emb, "X_emb_reduced"),
    (X_rif_reduced, y_rif, "X_rif_reduced")
]

# Dictionary to store the trained models for each dataset
trained_models = {}

for X, y, label in reduced_datasets:
    print(f"Evaluating dataset: {label}")

    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

    # Feature scaling
    if sparse.issparse(X):
        scaler = StandardScaler(with_mean=False)
    else:
        scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train).toarray() if sparse.issparse(X_train) else scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test).toarray() if sparse.issparse(X_test) else scaler.transform(X_test)

    # Reshape data for Conv1D layer
    X_train = X_train[..., np.newaxis]  # Add a new axis for the single channel
    X_test = X_test[..., np.newaxis]

    # Define the CNN model using Input layer
    model = Sequential([
        Input(shape=(X_train.shape[1], 1)),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Define early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

    # Train the model with a smaller batch size
    history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, 
                        callbacks=[early_stopping, reduce_lr], verbose=0)

    # Store the trained model in the dictionary
    trained_models[label] = model

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Set Accuracy: {accuracy:.3f}")

    # Make predictions
    y_pred_proba = model.predict(X_test).flatten()
    y_pred = (y_pred_proba >= 0.5).astype(int)

    # Calculate additional metrics
    print("Test Set Accuracy:", accuracy_score(y_test, y_pred))
    print("Test Set F1-Score:", f1_score(y_test, y_pred))
    print("Test Set ROC-AUC:", roc_auc_score(y_test, y_pred_proba))

    def sensitivity_specificity_score(y_true, y_pred):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        return sensitivity, specificity

    sensitivity, specificity = sensitivity_specificity_score(y_test, y_pred)
    print(f"Test Set Sensitivity: {sensitivity:.3f}")
    print(f"Test Set Specificity: {specificity:.3f}")

    # Clear the Keras session to free up memory
    K.clear_session()
    print("\n")

# trained_models dictionary now contains the trained model for each dataset
for label, model in trained_models.items():
    print(f"Trained model for {label} is stored.")


In [105]:
cnn_inh = trained_models['X_inh_reduced']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_inh_reduced, y_inh, test_size=0.2, random_state=1, stratify=y_inh)

# Feature scaling
if sparse.issparse(X):
    scaler = StandardScaler(with_mean=False)
else:
    scaler = StandardScaler()

X_train = scaler.fit_transform(X_train).toarray() if sparse.issparse(X_train) else scaler.fit_transform(X_train)
X_test = scaler.transform(X_test).toarray() if sparse.issparse(X_test) else scaler.transform(X_test)

# Reshape data for Conv1D layer
X_train = X_train[..., np.newaxis]  # Add a new axis for the single channel
X_test = X_test[..., np.newaxis]

# Make predictions
y_pred_prob = cnn_inh.predict(X_test).flatten()
y_pred = (y_pred_prob >= 0.5).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/cnn/inh_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/cnn/inh_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/cnn/inh_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/cnn/inh_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/cnn/inh_r/predicted_probability_distributions.png')
plt.close()

In [112]:
cnn_emb = trained_models['X_emb_reduced']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_emb_reduced, y_emb, test_size=0.2, random_state=1, stratify=y_emb)

# Feature scaling
if sparse.issparse(X):
    scaler = StandardScaler(with_mean=False)
else:
    scaler = StandardScaler()

X_train = scaler.fit_transform(X_train).toarray() if sparse.issparse(X_train) else scaler.fit_transform(X_train)
X_test = scaler.transform(X_test).toarray() if sparse.issparse(X_test) else scaler.transform(X_test)

# Reshape data for Conv1D layer
X_train = X_train[..., np.newaxis]  # Add a new axis for the single channel
X_test = X_test[..., np.newaxis]

# Make predictions
y_pred_prob = cnn_emb.predict(X_test).flatten()
y_pred = (y_pred_prob >= 0.5).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/cnn/emb_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/cnn/emb_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/cnn/emb_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/cnn/emb_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/cnn/emb_r/predicted_probability_distributions.png')
plt.close()

In [114]:
cnn_rif = trained_models['X_rif_reduced']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, recall_score, precision_recall_curve
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_rif_reduced, y_rif, test_size=0.2, random_state=1, stratify=y_rif)

# Feature scaling
if sparse.issparse(X):
    scaler = StandardScaler(with_mean=False)
else:
    scaler = StandardScaler()

X_train = scaler.fit_transform(X_train).toarray() if sparse.issparse(X_train) else scaler.fit_transform(X_train)
X_test = scaler.transform(X_test).toarray() if sparse.issparse(X_test) else scaler.transform(X_test)

# Reshape data for Conv1D layer
X_train = X_train[..., np.newaxis]  # Add a new axis for the single channel
X_test = X_test[..., np.newaxis]

# Make predictions
y_pred_prob = cnn_rif.predict(X_test).flatten()
y_pred = (y_pred_prob >= 0.5).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=['S', 'R'], yticklabels=['S', 'R'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig('plots/cnn/rif_r/cm.png')
plt.close()

# Classification Report
cr = classification_report(y_test, y_pred, output_dict=True)
cr_df = pd.DataFrame(cr).transpose()

# Replace '0' and '1' with 'S' and 'R'
cr_df.index = cr_df.index.map({'0': 'S', '1': 'R', 'accuracy': 'accuracy', 'macro avg': 'macro avg', 'weighted avg': 'weighted avg'})

# Plot and save Classification Report as a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(cr_df.iloc[:-1, :-1], annot=True, cmap='Blues', xticklabels=cr_df.columns[:-1], yticklabels=cr_df.index[:-1])
plt.title('Classification Report')
plt.savefig('plots/cnn/rif_r/cr.png')
plt.close()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot and save ROC Curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('plots/cnn/rif_r/roc_curve.png')
plt.close()

# Plot Recall and Accuracy Scores against probability thresholds
thresholds = np.arange(0.0, 1.1, 0.1)
recalls = []
accuracies = []

for threshold in thresholds:
    y_pred_threshold = (y_pred_prob >= threshold).astype(int)
    recalls.append(recall_score(y_test, y_pred_threshold))
    accuracies.append(accuracy_score(y_test, y_pred_threshold))

plt.figure(figsize=(10, 6))
plt.plot(thresholds, recalls, label='Recall', color='green', linestyle='-')
plt.plot(thresholds, accuracies, label='Accuracy', color='blue', linestyle='-')
plt.axvline(x=0.1, color='black', linestyle=':', linewidth=2)

plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Recall and Accuracy Scores')
plt.legend()
plt.savefig('plots/cnn/rif_r/recall_accuracy_thresholds.png')
plt.close()

# Density plot for predicted probability distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(y_pred_prob[y_test == 0], label='Susceptible', shade=True, color='green')
sns.kdeplot(y_pred_prob[y_test == 1], label='Resistant', shade=True, color='blue')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distributions')
plt.legend()
plt.savefig('plots/cnn/rif_r/predicted_probability_distributions.png')
plt.close()